# MIE1621 Fall 2017 Project
## Jonathan Smith 999694521

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy as sci
from numpy.linalg import inv,norm
from scipy.optimize import minimize

### Data Initialization & Pre-Processing
Create functions that will convert a csv file into 

In [2]:
### Manually input stock data
# stocks = pd.DataFrame(columns=['Return', 'var_1', 'var_2', 'var_3'],
#                    data=[[0.1073,0.02778,0.00387,0.00021],
#                         [0.0731,0.00387,0.01112,-0.0002],
#                         [0.0621,0.00021,-0.0002,0.00115]]
#                   )
# stocks.to_csv('stock_data.csv')

In [4]:
stocks = pd.read_csv('stock_data.csv')

In [6]:
def split (df):
    '''Function to split input dataframe into variance and return'''
    ret=df.loc[:,'Return'].as_matrix()
    var=df.iloc[:,2:].as_matrix()
    return ret, var

In [7]:
def kkt_convert (df,d=3.5):
    '''Function to split input dataframe into variance and return
    and convert them into kkt matrices'''
    ret=df.loc[:,'Return'].as_matrix()
    ret = np.append(ret,1)
    
    var=df.iloc[:,2:].as_matrix()
    var=d*var
    size=len(var)
    var = np.insert(var,size,1,axis=1)
    var = np.insert(var,size,1,axis=0)
    var[size][size]=0
    return ret, var 

In [8]:
ret_kkt, var_kkt = kkt_convert(stocks)

In [9]:
ret_kkt

array([ 0.1073,  0.0731,  0.0621,  1.    ])

In [10]:
var_kkt

array([[  9.72300000e-02,   1.35450000e-02,   7.35000000e-04,
          1.00000000e+00],
       [  1.35450000e-02,   3.89200000e-02,  -7.00000000e-04,
          1.00000000e+00],
       [  7.35000000e-04,  -7.00000000e-04,   4.02500000e-03,
          1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00]])

In [11]:
ret, var = split(stocks)

In [12]:
ret

array([ 0.1073,  0.0731,  0.0621])

In [13]:
var

array([[ 0.02778,  0.00387,  0.00021],
       [ 0.00387,  0.01112, -0.0002 ],
       [ 0.00021, -0.0002 ,  0.00115]])

### Define Functions to Evaluate Objective Function & Derivatives

In [14]:
def f(x, d=3.5):
    '''Return value of function'''
#     return np.dot(x,ret)-np.dot(0.5*d*np.matmul(np.transpose(x),var),x)
    return np.dot(x,ret)-np.dot(0.5*d*np.matmul(x,var),x)

In [15]:
def df_kkt(x): 
    '''return value of kkt gradient'''
    return np.matmul(var_kkt,x)-ret_kkt

In [16]:
df_kkt([1,1,1,1])

array([ 1.00421 ,  0.978665,  0.94196 ,  2.      ])

In [17]:
# minimize(f,[1,1,1],method='BFGS')

In [18]:
# def d_f(x,d=4):
#     return ret-d*np.matmul(var,x)

In [19]:
# def d2_f(x,d=4):
#     return -d*var

In [20]:
f([1,1,1])

0.15883249999999999

### Define Functions for Newton, SDM, BFGS Methods

In [21]:
x=[1,1,1,1]
alpha=1
d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
x_k = x - alpha*d
Rho=0.5
gamma=0.0001

print(f(x[:-1]))
print(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))

0.1588325
0.159215233116


In [22]:
def newton (df,d,init,tol=0.001,limit=100, backtrack=False, Rho=0.5, gamma=0.01):
    ret,var=split(df)
    x=init
    rho=Rho
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
        x_k = x - alpha*d
        while (alpha>0.01)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
            alpha=alpha*rho
            x_k = x - alpha*d
            print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
#     print(x)
    print('Solution: x =',x,', Return:',f(x))

In [60]:
def steepest_descent (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.01):
    ret,var=split(df)
    x=init
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(np.all(df_kkt(x)<0))&(i<limit):
        alpha=1
        d=df_kkt(x)
        x_k = x - alpha*d
        while (alpha>0.01)&(backtrack==True)&((f(x_k[:-1]))>(f(x[:-1])+alpha*gamma*np.matmul(d,(df_kkt(x))**2))):
            alpha=alpha*rho
            x_k = x - alpha*d
            print('alpha: ',alpha)
        x_k = x-alpha*d
        delta = x_k-x
#         print(delta)
        x=x_k
        i+=1
        print('Iteration',i,':',x)
        print('slope:',df_kkt(x))
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

In [24]:
def BFGS (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.0001):
    ret,var=split(df)
    x=init
    H=inv(np.eye(1+len(var)))
    I=np.eye(1+len(var))
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(np.asarray(H),df_kkt(x))
        x_k = x - alpha*d
        while (alpha>0.1)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
#             print('stop',alpha*norm(d))
            alpha=alpha*rho
            x_k = x - alpha*d
            print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
#         find udpated H inverse
        s=np.asmatrix(x_k-x)
        y=np.asmatrix(df_kkt(x_k)-df_kkt(x))
#         a=1/(np.matmul(np.transpose(y),s))
        a=np.asscalar(1/(y*np.transpose(s)))
#         A=a*np.matmul(s,np.transpose(y))
        A=a*(np.transpose(s)*y)
#         B=a*np.matmul(y,np.transpose(s))
        B=a*(np.transpose(y)*s)
#         C=a*np.matmul(s,np.transpose(s))
        C=a*(np.transpose(s)*(s))
        H=np.matmul(np.matmul((I-A),H),(I-B))+C 
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

### Perform 3 Methods with Step Length = 1

In [25]:
ret, var = split(stocks)
newton(stocks,4,[1,1,1,1])

Initial guess:  [1, 1, 1, 1]
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [ 0.  0.  0.  0.]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [53]:
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
steepest_descent(stocks,4,[0.4,0.1,0.4,0.01])

Initial guess:  [0.4, 0.1, 0.4, 0.01]
Iteration 1 : [ 0.4567595  0.15407    0.450266   0.11     ]
slope: [ 0.04952855  0.04876803  0.04994019  0.0610955 ]
Iteration 2 : [ 0.40723095  0.10530197  0.40032581  0.0489045 ]
slope: [-0.01707988 -0.01486143 -0.01135859 -0.08714127]
Iteration 3 : [ 0.42431083  0.12016341  0.4116844   0.13604577]
slope: [ 0.07193171  0.07308164  0.07583055 -0.04384137]
Iteration 4 : [ 0.35237912  0.04708177  0.33585385  0.17988713]
slope: [ 0.10773353  0.11315743  0.11936499 -0.26468526]
Iteration 5 : [ 0.24464559 -0.06607566  0.21648886  0.4445724 ]
slope: [ 0.36032341  0.37206291  0.38356983 -0.60494121]
Iteration 6 : [-0.11567782 -0.43813858 -0.16708097  1.04951361]
slope: [ 0.92490886  0.95791136  0.98696278 -1.72089737]
Iteration 7 : [-1.04058668 -1.39604993 -1.15404375  2.77041098]
slope: [ 2.54217701  2.6296898   2.70387835 -4.59068037]
Iteration 8 : [-3.5827637  -4.02573973 -3.85792211  7.36109134]
slope: [  6.84807501   7.08548156   7.28364789 -12.4664

In [27]:
BFGS(stocks,4,[1,1,1,1])

Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
Iteration 2 : [ 0.60147834  0.60766434  0.6395555  -0.5334302 ]
Iteration 3 : [ 2.01458584  2.00211579  2.05697165 -1.36729661]
Iteration 4 : [ 0.37926823  0.36520077  0.38977057  0.08642783]
Iteration 5 : [ 0.33674282  0.31035651  0.33944003  0.05852694]
Iteration 6 : [ 0.34756647  0.30907218  0.34196769  0.06184588]
Iteration 7 : [ 0.37033821  0.29716183  0.34134922  0.06434301]
Iteration 8 : [ 0.41764051  0.26367232  0.33493663  0.06539792]
Iteration 9 : [ 0.45653346  0.22881726  0.32619087  0.06305928]
Iteration 10 : [ 0.46733115  0.21358531  0.32220453  0.06030821]
Iteration 11 : [ 0.46509935  0.21197629  0.32282827  0.0594258 ]
Iteration 12 : [ 0.46282827  0.21104822  0.3247461   0.05912969]
Iteration 13 : [ 0.45806373  0.2065993   0.33197519  0.05876375]
Iteration 14 : [ 0.45312737  0.19669375  0.34563295  0.05873498]
Iteration 15 : [ 0.45082132  0.18248035  0.36307494  0.05932425]
Iteration 16 

### Perform 3 Methods with Backtracking

In [28]:
newton(stocks,4,[1,1,1,1], backtrack=True)

Initial guess:  [1, 1, 1, 1]
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
alpha:  0.03125
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [  8.32667268e-17  -2.22044605e-16  -1.17961196e-16  -2.66453526e-15]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [63]:
steepest_descent(stocks,4,[1,1,1,1], backtrack=True, tol=0.0001)

Initial guess:  [1, 1, 1, 1]
slope: [ 1.00421   0.978665  0.94196   2.      ]
Solution: x = [1, 1, 1] , Return: 0.1588325


In [30]:
BFGS(stocks,4,[1,1,1,1], backtrack=True, tol=0.001)

Initial guess:  [1, 1, 1, 1]
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 2 : [ 0.03364552  0.05798058  0.09438472 -0.97083939]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 3 : [ 0.15745429  0.17948903  0.2170464  -0.99561796]
Iteration 4 : [ 8.16394517  7.93417629  7.99447414  1.18320926]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 5 : [ 7.69597509  7.47886257  7.53735717  1.10615406]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 6 : [ 7.24234883  7.0365261   7.09340237  1.03963153]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 7 : [ 6.81304898  6.61710858  6.67268054  0.97826963]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 8 : [ 6.4099826   6.22258186  6.27717223  0.92093946]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 9 : [ 6.03227662  5.85218358  5.90609529  0.86722678]
alpha:  0.5
alpha:  0.25


### Scale up problem

In [31]:
stocks_2 = pd.read_csv('stock_data_2.csv')
stocks_2

,Unnamed: 0,Return,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10
0,0,146.469681,253.982693,1082.215766,110.719016,45.171571,301.860779,524.349411,-20.831911,2.508390,-52.108590,1344.859458
1,1,903.057530,1082.215766,5760.202676,569.708517,336.947639,1395.358573,2415.934209,-56.261749,12.895522,-246.513619,6546.499421
2,2,76.860558,110.719016,569.708517,65.213668,27.665204,159.999897,279.735187,-11.545475,1.193145,-26.557247,710.784809
3,3,94.038482,45.171571,336.947639,27.665204,301.652075,48.916565,86.887898,2.802558,0.976607,-13.796520,239.968392
4,4,283.091554,301.860779,1395.358573,159.999897,48.916565,464.537101,808.476811,-51.188807,3.505359,-74.143305,1748.498986
5,5,205.560718,524.349411,2415.934209,279.735187,86.887898,808.476811,1463.017292,-98.090319,6.828543,-134.659310,2988.909280
6,6,103.744900,-20.831911,-56.261749,-11.545475,2.802558,-51.188807,-98.090319,17.677925,-0.378214,7.932859,-74.330119
7,7,2.426096,2.508390,12.895522,1.193145,0.976607,3.505359,6.828543,-0.378214,0.089915,-0.799359,11.448998
8,8,26.983626,-52.108590,-246.513619,-26.557247,-13.796520,-74.143305,-134.659310,7.932859,-0.799359,13.954512,-271.488660
9,9,835.000562,1344.859458,6546.499421,710.784809,239.968392,1748.498986,2988.909280,-74.330119,11.448998,-271.488660,9429.207518


In [32]:
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)

In [33]:
start=[1,1,1,1,1,1,1,1,1,1,1]

In [34]:
newton(stocks_2,4,start)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [  0.29812093   0.17875891  -1.44010243   0.16449225   2.69975876
   0.82754477   6.28669517 -20.2414079   12.58968767  -0.36354813
   8.77417019]
Iteration 2 : [  0.29812093   0.17875891  -1.44010243   0.16449225   2.69975876
   0.82754477   6.28669517 -20.2414079   12.58968767  -0.36354813
   8.77417019]
slope: [ -2.84217094e-14   4.54747351e-13   7.10542736e-13   8.52651283e-14
   6.82121026e-13   1.30739863e-12   9.94759830e-14  -1.15463195e-14
  -1.20792265e-13   0.00000000e+00  -2.33146835e-15]
Solution: x = [  0.29812093   0.17875891  -1.44010243   0.16449225   2.69975876
   0.82754477   6.28669517 -20.2414079   12.58968767  -0.36354813] , Return: 846.149025085


In [35]:
steepest_descent(stocks_2,4,start)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [ -1.24280734e+04  -6.14563968e+04  -6.52734797e+03  -3.67613349e+03
  -1.65372853e+04  -2.89893008e+04   1.09849128e+03  -1.31515071e+02
   2.82061096e+03  -7.85252527e+04  -8.00000000e+00]
Iteration 2 : [  6.87816324e+08   3.43114532e+09   3.62499846e+08   1.56676680e+08
   9.07939623e+08   1.56836250e+09  -4.67249553e+07   6.97435071e+06
  -1.48810907e+08   4.48482797e+09   2.04345203e+05]
Iteration 3 : [ -3.87505976e+13  -1.93277271e+14  -2.04225763e+13  -8.76141956e+12
  -5.11197070e+13  -8.82792745e+13   2.61375783e+12  -3.91760733e+11
   8.37184676e+12  -2.53009870e+14  -1.14105024e+10]
Iteration 4 : [  2.18445560e+18   1.08952784e+19   1.15126563e+18   4.93759256e+17
   2.88166236e+18   4.97632148e+18  -1.47304394e+17   2.20810273e+16
  -4.71906965e+17   1.42637095e+19   6.43015462e+14]
Iteration 5 : [ -1.23146079e+23  -6.14207635e+23  -6.49012304e+22  -2.78346888e+22
  -1.62450125e+23  -2.80533769e+23   8.30401831

/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in subtract
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in subtract
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in greater_equal


In [36]:
BFGS(stocks_2,4,start)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [ -1.24280734e+04  -6.14563968e+04  -6.52734797e+03  -3.67613349e+03
  -1.65372853e+04  -2.89893008e+04   1.09849128e+03  -1.31515071e+02
   2.82061096e+03  -7.85252527e+04  -8.00000000e+00]
Iteration 2 : [ -226.98233486  -591.54206575   -96.9994664   -897.14519794  -431.49716461
 -1168.52244226   269.72683636    -7.80007121   180.91544748  1031.40536722
    -4.37692246]
Iteration 3 : [  4.31138029e+05   2.65771041e+06   2.29895978e+05   1.26160504e+06
   1.14283276e+06   2.40404964e+06  -3.60618438e+05   2.50374115e+04
  -3.05419365e+05  -3.24211963e+06   1.94287085e+03]
Iteration 4 : [  36131.93071028 -408659.20544237   24963.5606823   339478.81141377
  227508.58981711  479687.15193279 -104931.38716111     956.54669333
  -28447.45173436   77510.26272774     527.97499048]
Iteration 5 : [ -6.32630355e+07   5.38062465e+08  -4.35272757e+07  -1.82022993e+08
  -3.84026219e+08  -7.86837440e+08   1.64984351e+08  -1.75938655e+06


In [37]:
newton(stocks_2,4,start, backtrack=True)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
alpha:  0.03125
alpha:  0.015625
alpha:  0.0078125
Iteration 1 : [ 0.99451657  0.99358405  0.9809367   0.9934726   1.01327937  0.99865269
  1.04130231  0.8340515   1.09054443  0.98934728  1.0607357 ]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
alpha:  0.03125
alpha:  0.015625
alpha:  0.0078125
Iteration 2 : [ 0.98907598  0.98721823  0.96202233  0.98699619  1.02645499  0.99731591
  1.08228194  0.66939947  1.18038149  0.97877778  1.12099691]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
alpha:  0.03125
alpha:  0.015625
alpha:  0.0078125
Iteration 3 : [ 0.98367789  0.98090214  0.94325573  0.98057037  1.03952767  0.99598958
  1.12294142  0.50603379  1.2695167   0.96829086  1.18078733]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
alpha:  0.03125
alpha:  0.015625
alpha:  0.0078125
Iteration 4 : [ 0.97832198  0.9746354   0.92463575  0.97419476  1.05249823  0.99467

In [38]:
steepest_descent(stocks_2,4,start, backtrack=True, tol=0.01)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [ -1.24280734e+04  -6.14563968e+04  -6.52734797e+03  -3.67613349e+03
  -1.65372853e+04  -2.89893008e+04   1.09849128e+03  -1.31515071e+02
   2.82061096e+03  -7.85252527e+04  -8.00000000e+00]
Iteration 2 : [  6.87816324e+08   3.43114532e+09   3.62499846e+08   1.56676680e+08
   9.07939623e+08   1.56836250e+09  -4.67249553e+07   6.97435071e+06
  -1.48810907e+08   4.48482797e+09   2.04345203e+05]
Iteration 3 : [ -3.87505976e+13  -1.93277271e+14  -2.04225763e+13  -8.76141956e+12
  -5.11197070e+13  -8.82792745e+13   2.61375783e+12  -3.91760733e+11
   8.37184676e+12  -2.53009870e+14  -1.14105024e+10]
Iteration 4 : [  2.18445560e+18   1.08952784e+19   1.15126563e+18   4.93759256e+17
   2.88166236e+18   4.97632148e+18  -1.47304394e+17   2.20810273e+16
  -4.71906965e+17   1.42637095e+19   6.43015462e+14]
Iteration 5 : [ -1.23146079e+23  -6.14207635e+23  -6.49012304e+22  -2.78346888e+22
  -1.62450125e+23  -2.80533769e+23   8.30401831

/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in subtract
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in subtract
/Users/Jonathan/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in greater_equal


In [39]:
BFGS(stocks_2,4,start, backtrack=True, tol=0.001)

Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Iteration 1 : [ -1.24280734e+04  -6.14563968e+04  -6.52734797e+03  -3.67613349e+03
  -1.65372853e+04  -2.89893008e+04   1.09849128e+03  -1.31515071e+02
   2.82061096e+03  -7.85252527e+04  -8.00000000e+00]
alpha:  0.5
alpha:  0.25
alpha:  0.125
alpha:  0.0625
Iteration 2 : [ -1.16655052e+04  -5.76523434e+04  -6.12545118e+03  -3.50244672e+03
  -1.55306735e+04  -2.72505021e+04   1.04669350e+03  -1.23782884e+02
   2.65562999e+03  -7.35529616e+04  -7.77355765e+00]
Iteration 3 : [  4.31138029e+05   2.65771041e+06   2.29895978e+05   1.26160504e+06
   1.14283276e+06   2.40404964e+06  -3.60618438e+05   2.50374115e+04
  -3.05419365e+05  -3.24211963e+06   1.94287085e+03]
Iteration 4 : [   963133.56160308  -8091065.41960947    635453.30883914
   7255792.94354662   5093356.55492205  10624090.28838268
  -2246605.41058263     19454.0522127    -626216.52204855   3745212.2314732
     11335.20151866]
Iteration 5 : [ -1.10562072e+08   5.79387632e+08  -7.4